In [1]:
import pandas as pd
import numpy as np
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait

import time
import datetime

import warnings
warnings.filterwarnings("ignore")

In [185]:
# https://chromedriver.chromium.org/downloads
local_path_of_driver = "/Users/steffenbrockmann/Downloads/chromedriver-2"

In [5]:
def check_refund(train_number):
    driver = webdriver.Chrome(executable_path=local_path_of_driver) 
    base_URL = 'https://italoinviaggio.italotreno.it/en/train'
    train_number = train_number
    driver.get(base_URL + '/' + str(train_number))
    try:
        delay = driver.find_element_by_xpath('/html/body/div[2]/section/div/div/div[1]/div/div/div[3]/span[2]').get_attribute("innerText")
        #print(delay)
        if (int(delay.split()[0])) >= 60:
            return True
        else:
            return False
    except:
        #print('There is no delay')
        return False

In [6]:
check_refund(8907)

False

In [7]:
def search_ticket_str(departure_station, arrival_station, dep_hour = 0):
    
    train_URL = 'https://www.italotreno.it/en/destinations-timetable/trains-schedules'    
    
    options = webdriver.ChromeOptions()
    options.add_experimental_option("excludeSwitches", ["enable-automation"])
    options.add_experimental_option('useAutomationExtension', False)

    driver = webdriver.Chrome(executable_path=local_path_of_driver, options=options)
    driver.get(train_URL)

    search_box_from = driver.find_element_by_xpath('/html/body/main/section[3]/div[2]/form/div[1]/div/table/tbody/tr[1]/td[1]/fieldset/div/input[1]').send_keys(departure_station)
    search_box_to = driver.find_element_by_xpath('/html/body/main/section[3]/div[2]/form/div[1]/div/table/tbody/tr[1]/td[3]/fieldset/div/input[1]').send_keys(arrival_station)

    button = driver.find_element_by_xpath('/html/body/main/section[3]/div[2]/form/div[1]/div/table/tbody/tr[1]/td[4]/div/a')
    webdriver.ActionChains(driver).click_and_hold(button).perform()
    driver.execute_script("arguments[0].click();", button)

    time.sleep(10)

    timing = []
    num_options = len(driver.find_elements_by_xpath("/html/body/main/section[3]/div[3]/div[2]/table/tbody/tr"))
    
    if num_options == 0:
        return f"Sorry, no routes available from {departure_station} to {arrival_station}"
    
    dep_hour = str(dep_hour)+"00"
    for i in range(1, num_options + 1):
        time_dep = driver.find_element_by_xpath(f"/html/body/main/section[3]/div[3]/div[2]/table/tbody/tr[{i}]/td[1]/p[1]").get_attribute("innerText")
        time_arr = driver.find_element_by_xpath(f"/html/body/main/section[3]/div[3]/div[2]/table/tbody/tr[{i}]/td[2]/p[1]").get_attribute("innerText")
        
        if (int(time_dep[:2]+time_arr[3:]) - int(dep_hour)) >= 0:
            timing.append((i, time_dep, time_arr))
    
    if len(timing) == 0:
        return f"Sorry, no available trains after {dep_hour[:-2]}:00"
    
    # If same departure time, first train is usually available on weekdays+saturdays, the second one on sundays
    x = 0
    if timing[0][1] == timing[1][1]:
        if (datetime.datetime.today().weekday() + 1) == 6:
            x = 1

    choice = timing[x][0]
    time_of_departure, time_of_arrival = timing[x][1:]
    train_number = driver.find_element_by_xpath(f'/html/body/main/section[3]/div[3]/div[2]/table/tbody/tr[{choice}]/td[4]/p[2]').get_attribute("innerText")
    
    button_book = driver.find_element_by_xpath('/html/body/main/section[3]/div[3]/div[2]/table/tbody/tr[1]/td[7]/div/div')
    driver.execute_script("arguments[0].click();", button_book)
    
    price = driver.find_element_by_xpath('/html/body/form/div/div[2]/div[6]/div[1]/div[2]/div/ul/div/div/li[2]/p[3]').get_attribute("innerText")
    today = datetime.date.today()
    date = (f'{today.year}-{today.month}-{today.day+1}')
    
    
    ticket = f"{date}_{departure_station}_{arrival_station}_{time_of_departure}_{time_of_arrival}_{train_number}_{price}"
       
    return ticket

In [8]:
search_ticket_str("Milano Centrale", "Roma Termini", 20)

'2021-12-14_Milano Centrale_Roma Termini_20:40_00:19_9963_89.90 €'